In [1]:
import aiohttp
import asyncio
import numpy as np
import ujson as json

In [2]:
def json_to_numpy(json_str, key='predictions'):
    return np.array(json_str[key])

def numpy_to_json(nd_array, key='instances'):
    try:
        if nd_array.shape[1] == 1:
            nd_array.reshape(nd_array.shape[0])
        nd_array_list =  { key : nd_array.tolist() } 
        return nd_array_list
    except Exception as e:
        nd_array_list =  { key : [] } 
        return nd_array_list

In [3]:
MODEL_HOST = 'http://192.168.64.13:30126'

In [4]:
async def async_predict(indata):    
    async with aiohttp.ClientSession() as session:
        async with session.post(MODEL_HOST + '/v1/models/model:predict', json=numpy_to_json(indata)) as resp:
            response_json = await resp.json()
            return json_to_numpy(response_json)

In [5]:
indata = np.random.randint(100000, size=(10000, 20)).astype(np.float32)

In [6]:
loop = asyncio.get_event_loop()
%time prediction = loop.run_until_complete(async_predict(indata))

CPU times: user 80.6 ms, sys: 17.3 ms, total: 98 ms
Wall time: 1.02 s


In [10]:
def test(nb_calls):
    for i in range(nb_calls):
        %time prediction = loop.run_until_complete(async_predict(indata))

In [ ]:
%time test(10)

CPU times: user 89 ms, sys: 19.1 ms, total: 108 ms
Wall time: 1.04 s
